In [72]:
import mujoco
import mujoco_viewer

# scene.xml includes the robot model and a simple environment
model = mujoco.MjModel.from_xml_path("../src/assets/unitree_h1/scene.xml")

# Contains the state of the model (time: .time, pos: .qpos, vel: .qvel) 
data = mujoco.MjData(model)

In [73]:
# Print out the name of the body accessors
try:
    model.geom()
except KeyError as e:
    print(f"model.geom: {e}")
    
try:
    data.geom()
except KeyError as e:
    print(f"data.geom: {e}")
    
try:
    data.body()
except KeyError as e:
    print(f"data.body: {e}")
    
    
trunk_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY.value, "forearm_link")
# print(data.body(trunk_id))

print(f"actuator control range: {model.actuator_ctrlrange}")

model.geom: "Invalid name ''. Valid names: ['floor', 'head', 'helmet', 'hip', 'left_shoulder', 'right_shoulder', 'torso']"
data.geom: "Invalid name ''. Valid names: ['floor', 'head', 'helmet', 'hip', 'left_shoulder', 'right_shoulder', 'torso']"
data.body: "Invalid name ''. Valid names: ['left_ankle_link', 'left_elbow_link', 'left_hip_pitch_link', 'left_hip_roll_link', 'left_hip_yaw_link', 'left_knee_link', 'left_shoulder_pitch_link', 'left_shoulder_roll_link', 'left_shoulder_yaw_link', 'pelvis', 'right_ankle_link', 'right_elbow_link', 'right_hip_pitch_link', 'right_hip_roll_link', 'right_hip_yaw_link', 'right_knee_link', 'right_shoulder_pitch_link', 'right_shoulder_roll_link', 'right_shoulder_yaw_link', 'target', 'torso_link', 'world']"
actuator control range: [[-40.  40.]
 [-40.  40.]
 [-18.  18.]
 [-18.  18.]]


In [74]:
print(f"Simulated time: {data.time} sec")
print(f"Default timestep: {model.opt.timestep} sec")
print(f"Degrees of freedom: {model.nv=}")

print(f"{len(data.qpos)=} {data.qpos=}")
print(f"{len(data.qvel)=} {data.qvel=}")
print(f"{len(data.xpos)=} {data.xpos=}")
print(f"{len(data.site_xpos)=} {data.site_xpos=}")

Simulated time: 0.0 sec
Default timestep: 0.002 sec
Degrees of freedom: model.nv=4
len(data.qpos)=4 data.qpos=array([0., 0., 0., 0.])
len(data.qvel)=4 data.qvel=array([0., 0., 0., 0.])
len(data.xpos)=22 data.xpos=array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
len(data.site_xpos)=3 data.site_xpos=array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])


In [75]:
import mujoco_viewer.mujoco_viewer
import numpy as np
import time
# Setup viewer
viewer = mujoco_viewer.MujocoViewer(model, data)
# Reset the simulation
mujoco.mj_resetDataKeyframe(model, data, 0)


print([mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR.value, i)
    for i in range(len(data.qfrc_actuator))])

dof_position_limit_multiplier = 0.9 # The % of the range that is not penalized
ctrl_range_offset = 0.5 * (1 - dof_position_limit_multiplier) * (model.actuator_ctrlrange[:, 1] - model.actuator_ctrlrange[:, 0])
_soft_ctrl_range = np.copy(model.actuator_ctrlrange)
_soft_ctrl_range[:, 0] += ctrl_range_offset
_soft_ctrl_range[:, 1] -= ctrl_range_offset

print(f"{model.jnt_range=}")

for _ in range(1000):
    if viewer.is_alive:
        mujoco.mj_step(model, data)

        print(f"{data.qfrc_actuator[-12:]=}")
        
        print("\n")
        print(data.act)
        data.ctrl[0] = 0.5
        print(data.xpos[-1])
        
        viewer.render()
        time.sleep(0.01)        

    else:
        break

viewer.close()

['right_shoulder_pitch', 'right_shoulder_roll', 'right_shoulder_yaw', 'right_elbow']
model.jnt_range=array([[-2.87,  2.87],
       [-3.11,  0.34],
       [-4.45,  1.3 ],
       [-1.25,  2.61]])
data.qfrc_actuator[-12:]=array([0., 0., 0., 0.])


[]
[ 0.3 -0.2  1.2]
data.qfrc_actuator[-12:]=array([ 1.99593003e+00, -6.14683634e-04, -5.49233457e-04, -5.89010813e-02])


[]
[ 0.3 -0.2  1.2]
data.qfrc_actuator[-12:]=array([ 1.96670033, -0.00203357, -0.00781266, -0.09502162])


[]
[ 0.3 -0.2  1.2]
data.qfrc_actuator[-12:]=array([ 1.93649633, -0.00371924, -0.01432457, -0.12847509])


[]
[ 0.3 -0.2  1.2]
data.qfrc_actuator[-12:]=array([ 1.90539051, -0.00563658, -0.02016251, -0.15954251])


[]
[ 0.3 -0.2  1.2]
data.qfrc_actuator[-12:]=array([ 1.87345011, -0.00775376, -0.02539545, -0.18847478])


[]
[ 0.3 -0.2  1.2]
data.qfrc_actuator[-12:]=array([ 1.84073771, -0.01004197, -0.03008471, -0.21549589])


[]
[ 0.3 -0.2  1.2]
data.qfrc_actuator[-12:]=array([ 1.80731173, -0.01247506, -0.03428483, -0.240

In [76]:
import numpy as np

a = np.array([3, 4])

print(np.linalg.norm(a))

5.0
